<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/recommender_topK_topP_minP_search/blob/main/article5_sampling_min_p_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Pre-processing ...

In [4]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# Load the dataset
data_cust_tckt = pd.read_csv('helpdesk_customer_tickets.csv')

data_cust_tckt_en = data_cust_tckt[data_cust_tckt['language'] == 'en']
data_cust_tckt_en.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
6,717,NaN,"Dear Tech Online Store Customer Support,\n\nI ...","Dear <name>,\n\nThank you for contacting Tech ...",Problem,Returns and Exchanges,low,en,Tech Online Store,Technical Support,Product Support,Returns and Exchanges,Network Issue,Hardware Failure,Problem Resolution,NaN,NaN,NaN
7,900,Frequent Disconnections and Crashes,Customer reports frequent disconnections and c...,We'll investigate the issue with Zoom 5.11.0. ...,Incident,Product Support,high,en,Software Development Company,Technical Support,Service Disruption,Network Issue,Software Bug,Problem Resolution,Urgent Issue,NaN,NaN,NaN
9,1032,Urgent: Immediate Assistance Required for Serv...,"Dear IT Services Customer Support, \n\nOur ser...","Dear <name>,\n\nThank you for contacting IT Se...",Problem,Technical Support,high,en,IT Services,Technical Support,IT Support,Urgent Issue,Service Disruption,System Crash,Problem Resolution,Incident Report,NaN,NaN
11,1438,Monitor Issue,"Hello Support Team, my Dell XPS 13 9310 is not...","Hello <name>,\nPlease try the following steps:...",Problem,Technical Support,medium,en,Tech Online Store,Technical Support,Product Support,Hardware Failure,Problem Resolution,Technical Guidance,NaN,NaN,NaN,NaN
17,1955,Issue Changing Status in Jira Software Project,"Dear Customer Support Team, \n\nI am unable to...","Dear <name>,\n\nThank you for reaching out. To...",Incident,Customer Service,medium,en,Software Development Company,Technical Support,Product Support,Problem Resolution,Software Bug,NaN,NaN,NaN,NaN,NaN


In [6]:
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

def remove_special_characters(text):
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

def to_lowercase(text):
    return text.lower()

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def preprocess_text(text):
    text = remove_html_tags(text)
    text = remove_special_characters(text)
    text = to_lowercase(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    text = text.strip()  # Remove leading/trailing whitespace
    return text



In [7]:
data = pd.DataFrame()
data['query'] = data_cust_tckt_en['body'].apply(preprocess_text)
data['resolution'] = data_cust_tckt_en['answer'].apply(preprocess_text)
data.head()

,query,resolution
6,dear tech online store customer support writin...,dear thank contacting tech online store custom...
7,customer report frequent disconnection crash v...,well investigate issue zoom 5110 meanwhile ens...
9,dear service customer support server affect ti...,dear thank contacting service customer support...
11,hello support team dell xps 13 9310 recognizin...,hello please try following step 1 ensure exter...
17,dear customer support team unable change issue...,dear thank reaching change issue status jira m...


In [8]:
user_query = ['JIRA software issue',
              'server issue',
              'windows os issue',
              'cisco router issue',
              'laptop hardware issue',
              'laptop software visual studio problem',
              'laptop software issue',
              'printer problem',
              'desktop monitor issue',
              'cloud cost problem']

#### Create embeddings of the user emails and the resolution emails

In [3]:
# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# get embedding df
data['q_emb'] = data['query'].apply(lambda x: model.encode(x, convert_to_tensor=True))
data['r_emb'] = data['resolution'].apply(lambda x: model.encode(x, convert_to_tensor=True))


#### Create functions to calculate cosine similarity, top-k, top-p and min-p search results

In [10]:
# calculate cosine similarity of user query with list of queries in db ...

def get_cos_sim(user_q, df):
  user_query_emb = model.encode(user_q, convert_to_tensor=True)
  cosine_scores = df['q_emb'].apply(lambda x: util.pytorch_cos_sim(user_query_emb, x)[0].cpu().numpy())
  return cosine_scores

# test
cosine_scores = get_cos_sim(user_q = user_query[0],
                            df = data)

In [11]:
# funtion to return top-k resolutions
def get_top_k(cos_scores, k, df, column):
  top_k_ind = np.argsort(cos_scores)[-k:][::-1]
  top_k_res = list(df[column].iloc[top_k_ind])
  return top_k_res

# test
get_top_k(cos_scores = cosine_scores,
          k = 3,
          df = data_cust_tckt_en,
          column = 'body') # change column to 'answer' to get the resolutions

['Dear Customer Support, Users are unable to access project management features in Jira Software after the recent update. We urgently need your assistance to resolve this issue. We are using Jira Software version 8.20. Please address this matter as soon as possible to limit disruption. Thank you, <name>',
 'Hello Customer Support Team,\n\nWe have encountered an urgent issue where Jira Software 8.20 is failing to initialize, severely disrupting our project management activities. We need immediate assistance to resolve this. Please address this as soon as possible.\n\nThank you,\n<name>',
 "Dear Customer Support,\n\nI'm unable to create new tickets in Jira Software 8.20 after the recent update. Could you please look into it urgently?\n\nBest regards,\n<name>"]

In [12]:
# function to get the top similar search results whose sum of similarity score is <= supplied p
def get_top_p(cos_scores, p, df, column):
  sorted_scores = pd.DataFrame()
  sorted_scores['sorted'] = cos_scores.sort_values(ascending = False)
  top_p_ind = sorted_scores[sorted_scores['sorted'].cumsum() < p].index.tolist()
  top_p_res = df[column].loc[top_p_ind].tolist()
  return top_p_res

# test
get_top_p(cos_scores = cosine_scores,
          p = 4.0,
          df = data_cust_tckt_en,
          column = 'body') # change column to 'answer' to get the resolutions

['Dear Customer Support, Users are unable to access project management features in Jira Software after the recent update. We urgently need your assistance to resolve this issue. We are using Jira Software version 8.20. Please address this matter as soon as possible to limit disruption. Thank you, <name>',
 'Hello Customer Support Team,\n\nWe have encountered an urgent issue where Jira Software 8.20 is failing to initialize, severely disrupting our project management activities. We need immediate assistance to resolve this. Please address this as soon as possible.\n\nThank you,\n<name>',
 "Dear Customer Support,\n\nI'm unable to create new tickets in Jira Software 8.20 after the recent update. Could you please look into it urgently?\n\nBest regards,\n<name>",
 'Dear Customer Support Team, \n\nI am unable to change the issue status in my Jira Software 8.20 project. It seems to require an administrative viewing configuration adjustment. Please assist me in resolving this issue at your ear

In [13]:
# following https://arxiv.org/pdf/2407.01082 sec. 3 eqn. 6
# a p_base is supplied to change the threshold of similarity dynamic according to the max similarity score

def get_min_p(cos_scores, p_base, df, column):
  sorted_scores = pd.DataFrame()
  sorted_scores['sorted'] = cosine_scores.sort_values(ascending = False)
  sorted_scores['sorted'] = sorted_scores['sorted'].apply(lambda x: x[0])
  p_thr = sorted_scores['sorted'].max()*p_base
  min_p_ind = sorted_scores[sorted_scores['sorted'] > p_thr].index.tolist()
  min_p_res = df[column].loc[min_p_ind].tolist()
  return min_p_res

# test
get_min_p(cos_scores = cosine_scores,
          p_base = 0.8,
          df = data_cust_tckt_en,
          column = 'body') # change column to 'answer' to get the resolutions

['Dear Customer Support, Users are unable to access project management features in Jira Software after the recent update. We urgently need your assistance to resolve this issue. We are using Jira Software version 8.20. Please address this matter as soon as possible to limit disruption. Thank you, <name>',
 'Hello Customer Support Team,\n\nWe have encountered an urgent issue where Jira Software 8.20 is failing to initialize, severely disrupting our project management activities. We need immediate assistance to resolve this. Please address this as soon as possible.\n\nThank you,\n<name>',
 "Dear Customer Support,\n\nI'm unable to create new tickets in Jira Software 8.20 after the recent update. Could you please look into it urgently?\n\nBest regards,\n<name>",
 'Dear Customer Support Team, \n\nI am unable to change the issue status in my Jira Software 8.20 project. It seems to require an administrative viewing configuration adjustment. Please assist me in resolving this issue at your ear

#### Summary generation

For ease of use, we use t5-large as summarizer. You can use any other summarizer of your choice.

In [ ]:
from transformers import pipeline

# Initialize summarization pipeline
summarizer = pipeline("summarization", model="t5-large")



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

#### Now, we will generate the summaries and save them ...
Parameters:
- k = 5 for top_k
- p = 4.0 for top_p
- p_base = 0.8

We will calculate summary for all the 10 queries and save both the resolutions it matched with and the summary.

In [ ]:
from tqdm import tqdm


all_res = []

for u_q in tqdm(user_query):
  print(f"Processing: {u_q}")

  # get cosine scores
  cosine_scores = get_cos_sim(user_q = u_q, df = data)

  # get similar resolutions
  top_k_gt_res = get_top_k(cos_scores = cosine_scores,
                        k = 5,
                        df = data_cust_tckt_en,
                        column = 'answer')
  top_p_gt_res = get_top_p(cos_scores = cosine_scores,
                        p = 4.0,
                        df = data_cust_tckt_en,
                        column = 'answer')
  min_p_gt_res = get_min_p(cos_scores = cosine_scores,
                        p_base = 0.8,
                        df = data_cust_tckt_en,
                        column = 'answer')

  # create summary of resolutions
  summary_top_k = summarizer(" ".join(top_k_gt_res), max_length=200, min_length=10, do_sample=True)
  summary_top_p = summarizer(" ".join(top_p_gt_res), max_length=200, min_length=10, do_sample=True)
  summary_min_p = summarizer(" ".join(min_p_gt_res), max_length=200, min_length=10, do_sample=True)

  # create dictionary of results
  all_res.append({'user_query': u_q,
                  'top_k_gt': top_k_gt_res,
                  'top_p_gt': top_p_gt_res,
                  'min_p_gt': min_p_gt_res,
                  'summary_top_k': summary_top_k[0]['summary_text'],
                  'summary_top_p': summary_top_p[0]['summary_text'],
                  'summary_min_p': summary_min_p[0]['summary_text']})

  print('Summarized resolutions')

# save in a data frame
all_res_df = pd.DataFrame(all_res)

  0%|          | 0/10 [00:00<?, ?it/s]

Processing: JIRA software issue


 10%|█         | 1/10 [01:22<12:20, 82.30s/it]

Summarized resolutions
Processing: server issue


 20%|██        | 2/10 [02:48<11:18, 84.76s/it]

...

Summarized resolutions
Processing: desktop monitor issue


 90%|█████████ | 9/10 [17:34<02:11, 132.00s/it]

Summarized resolutions
Processing: cloud cost problem


100%|██████████| 10/10 [19:07<00:00, 114.70s/it]

Summarized resolutions


In [ ]:
all_res_df.to_csv('all_results.csv', index = False)